# Plots the United States

It is always convenient to be able to plots states.  Let's boogie on dat.

In [8]:
%load_ext importable
from toolz.curried import *; from pandas import *; import pandas as pd

The importable extension is already loaded. To reload it, use:
  %reload_ext importable


## [Shape files from census.gov](https://www.census.gov/geo/maps-data/data/cbf/cbf_state.html)

## [`@aerispaha1`s gist to read shapefiles to a dataframe](https://gist.github.com/aerispaha/f098916ac041c286ae92d037ba5c37ba)

Load that file in with the `%load` magic.

In [9]:
# %load https://gist.githubusercontent.com/aerispaha/f098916ac041c286ae92d037ba5c37ba/raw/bbec58b3e39ee709e5e96aa65e87e6b568ab4a84/read_shapefile.py
def read_shapefile(shp_path):
	"""
	Read a shapefile into a Pandas dataframe with a 'coords' column holding
	the geometry information. This uses the pyshp package
	"""
	import shapefile

	#read file, parse out the records and shapes
	sf = shapefile.Reader(shp_path)
	fields = [x[0] for x in sf.fields][1:]
	records = sf.records()
	shps = [s.points for s in sf.shapes()]

	#write into a dataframe
	df = pd.DataFrame(columns=fields, data=records)
	df = df.assign(coords=shps)

	return df

In [10]:
from sklearn import preprocessing
from bokeh.plotting import *; from bokeh.models import *
output_notebook(resources=__import__('bokeh').resources.CDN)

Loading BokehJS ...

* Read the shapefile after unzipping the contents from census.gov

In [12]:
df = read_shapefile('../../../Downloads/cb_2016_us_state_500k/cb_2016_us_state_500k.shp')

# nth controls the density of edge points.
nth = 15

df = concat([
    df, concat(dict(
        x=df.coords.apply(compose(list, take_nth(nth), pluck(0))),
        y=df.coords.apply(compose(list, take_nth(nth), pluck(1)))
    ), axis=1)], axis=1).set_index('NAME')

for _ in 'xy': df[_] = df[_].apply(preprocessing.minmax_scale, args=[[-.5, .5]])

In [13]:
source = df[list('xy')].pipe(ColumnDataSource)

In [14]:
p = figure()
for i in range(50):
    p.patches(
        dict(
            field='x',
            transform=CustomJSTransform(v_func="""return xs.map(function(v, i){
                return v.map(function(w){return w+%i});
            });"""%i)
        ),
        dict(
            field='y', 
            transform=CustomJSTransform(
                v_func="""return xs.map(function(v, i){
                return v.map(function(w){return (w+i+%i) %s 50 });
            });"""%(i, '%'))
        ), fill_alpha=0, source=source)

show(p)

In [15]:
p = figure()
for i in range(50):
    p.patches(
        dict(
            field='x',
            transform=CustomJSTransform(v_func="""return xs.map(function(v, i){
                return v.map(function(w){return w+%i});
            });"""%i)
        ),
        dict(
            field='y', 
            transform=CustomJSTransform(
                v_func="""return xs.map(function(v, i){
                return v.map(function(w){return (w+i+%i) %s 50 });
            });"""%(i*0, '%'))
        ), fill_alpha=0, source=source)

show(p)

> Consider making a package that has each patch easily available with the correct transforms.